# Create Simple Dataset

In [6]:
import cv2
import os

In [7]:
card_simple_rect = (30, 50, 100, 200)
original_database_foldername = "resized"
name_database_foldername = "names"
suit_database_foldername = "suits"
img_type = "png"

In [8]:
def extract_card_name_suit_images(card_img):
    x1, y1, x2, y2 = card_simple_rect
    h = y2 - y1
    card_img_simple = card_img[y1:y2, x1:x2]
    name_img = card_img_simple[:h//2,:]
    suit_img = card_img_simple[h//2+1:,:]
    return name_img, suit_img

In [9]:
def extract_card_name_suit(filename):
    #suit_legend = {"C": "Clubs", "D": "Diamonds", "H": "Hearts", "S": "Spades"}
    name = filename.split(".")[0]
    name_split = name.split("_")
    return name_split[0], name_split[1]

In [11]:
# Create folder to store images related to names of the cards
if not os.path.exists(name_database_foldername):
   os.makedirs(name_database_foldername)

# Create folder to store images related to suits of the cards
if not os.path.exists(suit_database_foldername):
   os.makedirs(suit_database_foldername)

filenames = os.listdir(original_database_foldername)
for filename in filenames:
    name, suit = extract_card_name_suit(filename)
    img = cv2.imread(f"./{original_database_foldername}/{filename}")
    name_img, suit_img = extract_card_name_suit_images(img)
    if (not os.path.exists(f"{name_database_foldername}/{name}.{img_type}")):
        cv2.imwrite(f"{name_database_foldername}/{name}.{img_type}", name_img)
    if (not os.path.exists(f"{suit_database_foldername}/{suit}.{img_type}")):
        cv2.imwrite(f"{suit_database_foldername}/{suit}.{img_type}", suit_img)